In [4]:
import requests
import json
import pandas as pd
import numpy as np

!pip install quandl
import quandl

You should consider upgrading via the '/Users/Lex/anaconda3/bin/python -m pip install --upgrade pip' command.


In [5]:
#S&P500 Data Call

url = "https://alpha-vantage.p.rapidapi.com/query"

querystring = {"function":"TIME_SERIES_DAILY","symbol":"SPY","outputsize":"full","datatype":"json"}
# swap out compact for full to get 20 year history

headers = {
    'x-rapidapi-key': "85073711famsh14c401fb1c62dc5p138f19jsnd902d78d96ca",
    'x-rapidapi-host': "alpha-vantage.p.rapidapi.com"
    }

results = requests.request("GET", url, headers=headers, params=querystring)

#print(results.text)

In [6]:
# S&P 500 Data

sp500=pd.DataFrame()
sp500 = pd.DataFrame.from_dict(results.json()["Time Series (Daily)"])

sp500 = sp500.T
sp500.columns = ['open', 'high', 'low', 'close', 'volume']

sp500['prevclose'] = sp500.close.shift(1).astype(float)
sp500['close'] = sp500['close'].astype(float)

sp500['pct_change_in_close'] = (sp500['close'] - sp500['prevclose']) / sp500['prevclose']*100

sp500.head()

,open,high,low,close,volume,prevclose,pct_change_in_close
1999-11-01,136.5000,137.0000,135.5625,135.5625,4006500,NaN,NaN
1999-11-02,135.9687,137.2500,134.5937,134.5937,6516900,135.5625,-0.714652
1999-11-03,136.0000,136.3750,135.1250,135.5000,7222300,134.5937,0.673360
1999-11-04,136.7500,137.3593,135.7656,136.5312,7907500,135.5000,0.761033
1999-11-05,138.6250,139.1093,136.7812,137.8750,7431500,136.5312,0.984244


In [7]:
# Pulling the Fed_Rate

fed_rate = quandl.get("FRED/DFF", authtoken="NPExo1wnSkPZTDxv2bCR")

fed_rate['Prev_Rate'] = fed_rate.Value.shift(1).astype(float)
fed_rate['Value'] = fed_rate['Value'].astype(float)

fed_rate['pct_change_in_fed_rate'] = (fed_rate['Value'] - fed_rate['Prev_Rate']) / fed_rate['Value']*100

fed_rate.head()

,Value,Prev_Rate,pct_change_in_fed_rate
Date,,,
1954-07-01,1.13,NaN,NaN
1954-07-02,1.25,1.13,9.600000
1954-07-03,1.25,1.25,0.000000
1954-07-04,1.25,1.25,0.000000
1954-07-05,0.88,1.25,-42.045455


In [8]:
# Pulling US Treasuries

ten_year = quandl.get("USTREASURY/REALYIELD", authtoken="NPExo1wnSkPZTDxv2bCR")
ten_year.columns = ['five_yr', 'seven_yr', 'ten_yr', 'twenty_yr', 'thirty_yr']



ten_year['prev_10_yr'] = ten_year.ten_yr.shift(1).astype(float)
ten_year['ten_yr'] = ten_year['ten_yr'].astype(float)

ten_year['pct_change_in_10_yr'] = (ten_year['ten_yr'] - ten_year['prev_10_yr']) / ten_year['ten_yr']*100

ten_year.head()

,five_yr,seven_yr,ten_yr,twenty_yr,thirty_yr,prev_10_yr,pct_change_in_10_yr
Date,,,,,,,
2003-01-02,1.75,2.24,2.43,NaN,NaN,NaN,NaN
2003-01-03,1.75,2.25,2.43,NaN,NaN,2.43,0.000000
2003-01-06,1.79,2.27,2.46,NaN,NaN,2.43,1.219512
2003-01-07,1.76,2.24,2.42,NaN,NaN,2.46,-1.652893
2003-01-08,1.68,2.14,2.29,NaN,NaN,2.42,-5.676856


In [9]:
sp500_and_fed_rate = pd.merge(sp500, fed_rate, left_index=True, right_index=True)
sp500_and_fed_rate_and_ten_year = pd.merge(sp500_and_fed_rate, ten_year, left_index=True, right_index=True)

combined = sp500_and_fed_rate_and_ten_year.reset_index()

combined.columns = ['Date', 'open', 'high', 'low', 'close', 'volume', 'prevclose', 'pct_change_in_close', 'fed_rate', 'prev_fed_rate', 'pct_change_in_fed_rate', '5_yr', '7_yr', '10_yr', '20_yr', '30_yr', 'prev_10_yr', 'pct_change_in_10_yr']

combined['Date'] = pd.to_datetime(combined['Date'], format='%Y%m%d', errors='ignore')
combined.head()

,Date,open,high,low,close,volume,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr
0,2003-01-02,88.8500,91.3000,88.5400,91.07,44516300,88.23,3.218860,1.30,1.16,10.769231,1.75,2.24,2.43,NaN,NaN,NaN,NaN
1,2003-01-03,90.9100,91.3800,90.5000,91.35,32222600,91.07,0.307456,1.12,1.30,-16.071429,1.75,2.25,2.43,NaN,NaN,2.43,0.000000
2,2003-01-06,91.2400,93.4900,91.1700,92.96,40984500,91.35,1.762452,1.22,1.12,8.196721,1.79,2.27,2.46,NaN,NaN,2.43,1.219512
3,2003-01-07,92.9000,93.3700,92.2000,92.73,38640400,92.96,-0.247418,1.20,1.22,-1.666667,1.76,2.24,2.42,NaN,NaN,2.46,-1.652893
4,2003-01-08,92.2000,92.4000,91.0500,91.39,38702200,92.73,-1.445056,1.29,1.20,6.976744,1.68,2.14,2.29,NaN,NaN,2.42,-5.676856


In [10]:
df_final = combined
combined.dtypes

Date                      datetime64[ns]
open                              object
high                              object
low                               object
close                            float64
volume                            object
prevclose                        float64
pct_change_in_close              float64
fed_rate                         float64
prev_fed_rate                    float64
pct_change_in_fed_rate           float64
5_yr                             float64
7_yr                             float64
10_yr                            float64
20_yr                            float64
30_yr                            float64
prev_10_yr                       float64
pct_change_in_10_yr              float64
dtype: object

In [2]:
# # Pulling in Fed dates

# df_dates = pd.read_csv('FOMC_meeting_dates_els.csv')
# df_dates['Date'] = pd.to_datetime(df_dates['Date'], format='%Y%m%d', errors='ignore')

# #df_dates = df_dates.set_index('Date')

# df_dates.head()

In [ ]:
# df_final =  pd.merge(df_dates, combined, left_on='Date', right_on='Date')


In [11]:
# create a list of our conditions
conditions = [
    # S&P500 is positive

    # Positive, Negative, Positive = 1 - 1 + 1 = 1
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Negative, Negative = 1 - 1 - 1 = -1
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Positive, Neutral, #Positive = 1 + 0 + 1 = 2
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Neutral, #Negative = 1 + 0 - 1 = 0
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Positive, Positve, #Positive = 1 + 1 + 1 = 3
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Positive, Positive, #Negative = 1 + 1 - 1 = 2
    ((df_final['pct_change_in_close'] >= 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] < 0)),

    #S&P 500 is negative

    # Negatve, Negative, Positive = -1 - 1 + 1 = -1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Negative, Negative = -1 - 1 - 1 = -3
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] < 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Negatve, Neutral, #Positive = -1 + 0 + 1 = 0
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Neutral, #Negative = -1 + 0 - 1 = -2
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] == 0) & (df_final['pct_change_in_10_yr'] < 0)),

    # Negatve, Positve, #Positive = -1 + 1 + 1 = 1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] >= 0)),

    # Negatve, Positive, #Negative = -1 + 1 - 1 = -1
    ((df_final['pct_change_in_close'] < 0) & (df_final['pct_change_in_fed_rate'] > 0) & (df_final['pct_change_in_10_yr'] < 0)),
    
    ]

# create a list of the values we want to assign for each condition
values = ['1', '-1', '2', '0', '3', '2', '-1', '-3', '0', '-2', '1', '-1']

# create a new column and use np.select to assign values to it using our lists as arguments
df_final['label'] = np.select(conditions, values)

In [19]:
df_final.head()

,Date,open,high,low,close,volume,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr,label
0,2003-01-02,88.8500,91.3000,88.5400,91.07,44516300,88.23,3.218860,1.30,1.16,10.769231,1.75,2.24,2.43,NaN,NaN,NaN,NaN,0
1,2003-01-03,90.9100,91.3800,90.5000,91.35,32222600,91.07,0.307456,1.12,1.30,-16.071429,1.75,2.25,2.43,NaN,NaN,2.43,0.000000,1
2,2003-01-06,91.2400,93.4900,91.1700,92.96,40984500,91.35,1.762452,1.22,1.12,8.196721,1.79,2.27,2.46,NaN,NaN,2.43,1.219512,3
3,2003-01-07,92.9000,93.3700,92.2000,92.73,38640400,92.96,-0.247418,1.20,1.22,-1.666667,1.76,2.24,2.42,NaN,NaN,2.46,-1.652893,-3
4,2003-01-08,92.2000,92.4000,91.0500,91.39,38702200,92.73,-1.445056,1.29,1.20,6.976744,1.68,2.14,2.29,NaN,NaN,2.42,-5.676856,-1


In [13]:
df_final.groupby(['label']).mean()

,close,prevclose,pct_change_in_close,fed_rate,prev_fed_rate,pct_change_in_fed_rate,5_yr,7_yr,10_yr,20_yr,30_yr,prev_10_yr,pct_change_in_10_yr
label,,,,,,,,,,,,,
-1,138.993001,139.508694,-0.367889,1.816272,1.825532,-2.814254,0.871642,1.138600,1.368048,1.625913,1.023608,1.386003,-inf
-2,200.242298,201.781128,-0.785535,0.811979,0.811979,0.000000,0.167191,0.409298,0.600915,0.935821,0.990711,0.631745,-inf
-3,143.545509,144.858175,-0.899025,1.884912,1.939719,-10.144617,0.931930,1.204772,1.417158,1.647449,1.125842,1.460632,-inf
0,200.826842,200.947931,-0.048960,0.808658,0.808406,0.038617,0.043158,0.279294,0.484581,0.846477,0.890942,0.485657,-inf
1,140.851010,140.600411,0.164960,1.882757,1.891113,-1.374842,0.923185,1.184829,1.413647,1.632327,1.007437,1.382928,6.843716
2,188.480514,187.176350,0.710018,1.124055,1.113571,1.384243,0.350198,0.592038,0.792226,1.061840,0.945075,0.782453,inf
3,144.486966,143.457942,0.768614,2.005356,1.963668,6.059819,0.947282,1.206596,1.424301,1.669039,1.046719,1.391953,inf


In [20]:
df_final.to_csv('/Users/Lex/Documents/Berkeley/266_nlp/266FinalProject_v2/266FinalProject/data/Market_data/market_data_labels_by_date.csv', index = False, header=True)